In [3]:
! pip install lxml

                                              0.0/3.8 MB ? eta -:--:--
     -                                        0.2/3.8 MB 5.0 MB/s eta 0:00:01
     ---                                      0.4/3.8 MB 5.6 MB/s eta 0:00:01
     ------                                   0.6/3.8 MB 4.8 MB/s eta 0:00:01
     ---------                                0.9/3.8 MB 5.1 MB/s eta 0:00:01
     ------------                             1.1/3.8 MB 5.2 MB/s eta 0:00:01
     ----------------                         1.5/3.8 MB 5.8 MB/s eta 0:00:01
     -------------------                      1.8/3.8 MB 5.8 MB/s eta 0:00:01
     ---------------------                    2.0/3.8 MB 5.9 MB/s eta 0:00:01
     --------------------------               2.5/3.8 MB 6.3 MB/s eta 0:00:01
     ------------------------------           2.9/3.8 MB 6.3 MB/s eta 0:00:01
     ---------------------------------        3.2/3.8 MB 6.4 MB/s eta 0:00:01
     -------------------------------------    3.6/3.8 MB 6.5 MB/s eta 0

In [1]:
# Imports
import requests
from bs4 import BeautifulSoup
from termcolor import colored, cprint
import sys
from datetime import datetime, timedelta

In [13]:
#vars
stories = ['null']

def get_soup(url):
	response = requests.get(url)
	soup = BeautifulSoup(response.text, "html.parser")
	return soup

def get_headlines(item_start_id, links_to_fetch, category):
    base_url = "http://feeds.bbci.co.uk/news/"

    if not category or category.lower() == "general":
        scrape_url = f"{base_url}rss.xml"
    elif category.lower() == "science":
        scrape_url = f"{base_url}science_and_environment/rss.xml"
    elif category.lower() == "entertainment":
        scrape_url = f"{base_url}entertainment_and_arts/rss.xml"
    else:
        scrape_url = f"{base_url}{category.lower()}/rss.xml"
    
    print(f"Scraping {scrape_url} for the latest news...")
    current_item = item_start_id
    # Request and parse XML feed
    response = requests.get(scrape_url)
    soup = BeautifulSoup(response.content, "xml")
    # Get the current time
    current_time = datetime.now()
    # Iterate over the articles in the feed
    for item in soup.find_all("item")[:links_to_fetch]:
        pub_date = datetime.strptime(item.find("pubDate").text, "%a, %d %b %Y %H:%M:%S %Z")
        # Calculate the time difference between the current time and the publication time
        time_difference = current_time - pub_date
        hours_difference = time_difference.total_seconds() / 3600
        if hours_difference <= 5:
            article_url = item.link.text
            article_headline = item.title.text
            stories.append(article_url)
            cprint(f"[{current_item}]{article_headline}:", 'red', attrs=['bold'], end='\n')
            current_item += 1

In [16]:
class Body:
    def __init__(self, url: str):
        article = requests.get(url)
        self.soup = BeautifulSoup(article.content, "html.parser")
        self.body = self.get_body()
        self.title = self.get_title()

    def get_body(self) -> list:
        body_divs = self.soup.find_all('div', {'data-component': 'text-block'})
        if body_divs:
            body = []
            for div in body_divs:
                paragraphs = div.find_all('p')
                for p in paragraphs:
                    body.append(p.text)
            return body
        return []

    def get_title(self) -> str:
        title_element = self.soup.find("h1")
        return title_element.text.strip() if title_element else ""

In [24]:
def main():
	category = input("Enter a category (World, UK, Business, Politics, Health, Education, Science, Technology, Entertainment) or leave blank for general news: ")
	get_headlines(1, 10, category)
	article_id = input("select an article id or Q to quit\n")
	if article_id.lower() == 'q':
		cprint("Goodbye!", 'red', end='\n')
		sys.exit()

	else:
		article_url = stories[int(article_id)]
		print("art url", article_url)
		# get_article_text(article_url, 32)
		parsed = Body(article_url)
		body = ' '.join(parsed.body)
		print(body)

while __name__ == "__main__":
	main()


Scraping http://feeds.bbci.co.uk/news/world/rss.xml for the latest news...
[1]S Korea floods leave at least 20 dead as thousands forced to evacuate:
art url https://www.bbc.co.uk/news/business-66203677?at_medium=RSS&at_campaign=KARANGA
A scheme designed to help people who missed out on last winter's £400 energy bill subsidy was a "staggering failure", a senior MP has said. The Energy Bill Support Scheme Alternative Funding was set up for households who do not have an energy supplier, such as those in park homes. Nearly a million households could apply but only a fraction received the money. MP Angus MacNeil said the government should reopen the scheme, saying it had "missed the most vulnerable". The government said it had spent more than £50m supporting 130,000 households without a domestic energy supplier. All UK households were eligible for the £400 help with fuel bills, after energy prices rose sharply last year. For households who pay their bills by direct debit, the support was gi

SystemExit: 